# Digits
https://hackernoon.com/16-best-sklearn-datasets-for-building-machine-learning-models

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [2]:
if torch.cuda.is_available():
    print("CUDA (GPU support) is available in PyTorch!")
    device = torch.device("cuda")
else:
    print("CUDA (GPU support) is not available. Using CPU.")
    device = torch.device("cpu")

CUDA (GPU support) is available in PyTorch!


In [3]:
from sklearn.datasets import load_digits

# Load the digits dataset
digits = load_digits()

# Print the features and target data
print(digits.data)
print("digits.data.shape:",digits.data.shape)
print(digits.target)
print("digits.target:",digits.target.shape)

Note: you may need to restart the kernel to use updated packages.
[[ 0.  0.  5. ...  0.  0.  0.]
 [ 0.  0.  0. ... 10.  0.  0.]
 [ 0.  0.  0. ... 16.  9.  0.]
 ...
 [ 0.  0.  1. ...  6.  0.  0.]
 [ 0.  0.  2. ... 12.  0.  0.]
 [ 0.  0. 10. ... 12.  1.  0.]]
digits.data.shape: (1797, 64)
[0 1 2 ... 8 9 8]
digits.target: (1797,)


In [4]:
X = digits.images
y = digits.target
# Flatten the image data and scale
X = X.reshape(X.shape[0], -1)
X = StandardScaler().fit_transform(X)

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Convert to PyTorch tensors
X_train, X_test, y_train, y_test = map(
    torch.tensor, (X_train, X_test, y_train, y_test)
)

In [5]:
# Create Tensor datasets
train_ds = TensorDataset(X_train.float(), y_train.long())
test_ds = TensorDataset(X_test.float(), y_test.long())

# Data loaders
train_loader = DataLoader(train_ds, batch_size=32, shuffle=True)
test_loader = DataLoader(test_ds, batch_size=32, shuffle=False)

In [6]:
# Define the MLP model using nn.Sequential
model = nn.Sequential(
    nn.Linear(64, 128),  # 64 input features (8x8 images), 128 outputs
    nn.ReLU(),
    nn.Linear(128, 64),  # 128 input features, 64 outputs
    nn.ReLU(),
    nn.Linear(64, 10)    # 64 input features, 10 outputs (digits 0-9)
).to(device)

# Loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [7]:
# Training loop
num_epochs = 15
for epoch in range(num_epochs):
    model.train()
    for inputs, targets in train_loader:
        inputs, targets = inputs.to(device), targets.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

    # Evaluation
    model.eval()
    with torch.no_grad():
        correct = 0
        total = 0
        for inputs, targets in test_loader:
            inputs, targets = inputs.to(device), targets.to(device)

            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += targets.size(0)
            correct += (predicted == targets).sum().item()

    accuracy = 100 * correct / total
    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {loss.item():.4f}, Accuracy: {accuracy:.2f}%')

print("Training complete")

Epoch 1/15, Loss: 1.1373, Accuracy: 78.89%
Epoch 2/15, Loss: 0.3163, Accuracy: 90.83%
Epoch 3/15, Loss: 0.2170, Accuracy: 94.72%
Epoch 4/15, Loss: 0.3110, Accuracy: 96.11%
Epoch 5/15, Loss: 0.0724, Accuracy: 96.94%
Epoch 6/15, Loss: 0.0427, Accuracy: 96.67%
Epoch 7/15, Loss: 0.0684, Accuracy: 97.50%
Epoch 8/15, Loss: 0.0386, Accuracy: 97.22%
Epoch 9/15, Loss: 0.0128, Accuracy: 97.78%
Epoch 10/15, Loss: 0.0167, Accuracy: 97.78%
Epoch 11/15, Loss: 0.0193, Accuracy: 97.50%
Epoch 12/15, Loss: 0.0060, Accuracy: 97.50%
Epoch 13/15, Loss: 0.0099, Accuracy: 97.50%
Epoch 14/15, Loss: 0.0052, Accuracy: 97.50%
Epoch 15/15, Loss: 0.0052, Accuracy: 97.50%
Training complete
